In [1]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, models

# for videos load 
def load_videos(video_dir, frame_size=(64, 64), max_frames=60):
    video_data = []
    labels = []
    
    for label_dir in os.listdir(video_dir):
        label_path = os.path.join(video_dir, label_dir)
        if os.path.isdir(label_path):
            label = label_dir
            for video_file in os.listdir(label_path):
                video_path = os.path.join(label_path, video_file)
                cap = cv2.VideoCapture(video_path)
                frames = []
                
                while len(frames) < max_frames:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frame = cv2.resize(frame, frame_size)
                    frames.append(frame)
                
                cap.release()
                
                while len(frames) < max_frames:
                    frames.append(np.zeros((frame_size[0], frame_size[1], 3), dtype=np.uint8))
                
                video_data.append(np.array(frames))
                labels.append(label)

    return np.array(video_data), np.array(labels)

video_dir = '../input/spin-ball'  
X, y = load_videos(video_dir)

# Preprocess labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X = X / 255.0

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

#the 3D CNN model
model = models.Sequential()

model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(60, 64, 64, 3)))
model.add(layers.MaxPooling3D((2, 2, 2)))

model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
model.add(layers.MaxPooling3D((2, 2, 2)))

model.add(layers.Conv3D(128, (3, 3, 3), activation='relu'))
model.add(layers.MaxPooling3D((2, 2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout for regularization

model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# classification Report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes, target_names=label_encoder.classes_))

# Save the model
model.save('video_classification_3dcnn_model.h5')

# Function to classify new videos
def classify_video(video_path, model, label_encoder, frame_size=(64, 64), max_frames=60):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)
    
    cap.release()
    
    while len(frames) < max_frames:
        frames.append(np.zeros((frame_size[0], frame_size[1], 3), dtype=np.uint8))
    
    frames = np.array(frames) / 255.0
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    
    predictions = model.predict(frames)
    predicted_class = np.argmax(predictions, axis=1)
    
    return label_encoder.inverse_transform(predicted_class)[0]

# Example usage
video_path = 'path_to_new_video.mp4' 
predicted_label = classify_video(video_path, model, label_encoder)
print(f'Predicted label: {predicted_label}')


FileNotFoundError: [Errno 2] No such file or directory: '../input/spin-ball'